In [1]:
import os
from collections import defaultdict
from tqdm import tqdm

fname="Books"
path = "/DATA/Recommendation/amazon/Books"

In [2]:
import json
def open_json(file_path):
    with open(file_path, "r") as f:
        dict_list = [json.loads(line[:-1]) for line in f.readlines()]
    return dict_list
meta = open_json(os.path.join(path, f"meta_{fname}.jsonl"))
data = open_json(os.path.join(path, f"{fname}.jsonl"))

In [15]:
interactions = []
max_user = []
for l in tqdm(data):
    asin = l["asin"]
    rev = l["user_id"]
    time = l["timestamp"]
    interactions.append((asin, rev, time))
    if rev =="AGSP6LSQK32SQEJO3YVVNACPWMSQ":
        max_user.append((asin, time))

100%|██████████| 29475453/29475453 [00:36<00:00, 809289.28it/s] 


In [17]:
max_user = sorted(max_user, key=lambda x: x[1])
max_user[:20]

[('0380777606', 1106682486000),
 ('0867196653', 1187447270000),
 ('0345491939', 1190676200000),
 ('0345493982', 1190750723000),
 ('0451220277', 1191118866000),
 ('0061031151', 1191205206000),
 ('1421514087', 1191528759000),
 ('0061136158', 1191877881000),
 ('045146107X', 1191946751000),
 ('0446618241', 1192056675000),
 ('1598168754', 1192135385000),
 ('0425201503', 1192150505000),
 ('044661579X', 1192454150000),
 ('1421514079', 1195592742000),
 ('1897299060', 1195932693000),
 ('0758213719', 1196644721000),
 ('1599983540', 1200686421000),
 ('142151477X', 1200731356000),
 ('1427803145', 1200738857000),
 ('1427803307', 1201497956000)]

In [6]:
interactions = set(interactions)
countU = defaultdict(lambda : 0)
for asin, rev, time in interactions:
    countU[rev] += 1

In [14]:
list(countU.values())[21350],list(countU.keys())[21350]

(3888, 'AGSP6LSQK32SQEJO3YVVNACPWMSQ')

In [10]:
import numpy as np
np.percentile(list(countU.values()),q=[0,25,50,75,100])
np.argmax(list(countU.values()))

21350

In [11]:
from collections import Counter
txt = open(os.path.join(path, f"train.txt"), "r").readlines()
train = dict()
counter = Counter()
train_items = list()
for line in tqdm(txt):
    u, history = line.rstrip().split(":")
    history = [int(i) for i in history.split(" ")]
    train_items.extend(history)
    u = int(u)
    train.update({u: history})

100%|██████████| 1150206/1150206 [00:04<00:00, 279350.21it/s]


In [12]:
txt = open(os.path.join(path, f"{fname}.txt"), "r").readlines()
total = defaultdict(list)
counter = Counter()
total_items = list()
for line in tqdm(txt):
    u, i = line.rstrip().split(" ")
    u = int(u)
    i = int(i)
    total[u].append(i)
    total_items.append(i)

100%|██████████| 12187673/12187673 [00:16<00:00, 749638.31it/s]


In [14]:
len(set(total_items))

980848

In [6]:
import numpy as np
train_avg_len = np.mean([len(i) for i in train.values()])
total_avg_len = np.mean([len(i) for i in total.values()])
train_avg_len, total_avg_len


(6.365173716708138, 10.100337294681186)

### Preprocess 재구성
1) 중복제거
2) interactions 수 5 미만 사용자, 아이템 제거


In [5]:
from tqdm import tqdm
unique_interactions = list()

for l in tqdm(data):
    asin = l["asin"]
    rev = l["user_id"]
    time = l["timestamp"]
    interactions.append((asin, rev, time))
unique_interactions = set(interactions)

100%|██████████| 29475453/29475453 [00:30<00:00, 962231.20it/s] 


In [10]:
# 중복 제거 완료
data_list = list()
countI, countU, countP = defaultdict(lambda : 0), defaultdict(lambda : 0), defaultdict(lambda : 0)
for l in tqdm(data):
    asin = l["asin"]
    rev = l["user_id"]
    time = l["timestamp"]
    key = f"{asin}-{rev}-{time}"
    countI[key]+=1
    if countI[key] >= 2:
        continue
    else:
        countU[rev] += 1
        countP[asin] += 1
        data_list.append(l)

  0%|          | 0/29475453 [00:00<?, ?it/s]

100%|██████████| 29475453/29475453 [01:32<00:00, 318046.71it/s]


In [14]:
len(data_list), sum(countU.values()), sum(countP.values())

(29142325, 29142325, 29142325)

In [1]:
import sys
sys.path.append("/home/doyooni303/experiments/LLMRec/ReLLMRec")

from src.dataset.data_preprocess import preprocess
fname = "Books"
folder = "../../data/amazon/Books"
preprocess(fname, "jsonl", folder, True,)

Loading jsonl files..


100%|██████████| 29142325/29142325 [03:12<00:00, 151477.68it/s]


Saved Books_meta_name_dict.json
1188598 969145
Saved Books.txt
Saved Books_usermap.json
Saved Books_itemmap.json


3) 최소 interaction 수 5 and train, valid, test 구분
- train user의 최소 interactions 수 3

In [8]:
from tqdm import tqdm
from collections import defaultdict
import os
usernum, itemnum = 0, 0
User = defaultdict(list)
f = tqdm(open(os.path.join(folder, f"{fname}.txt"), "r"))
for line in f:
    u, i = line.rstrip().split(" ")
    u = int(u)
    i = int(i)
    usernum = max(u, usernum)
    itemnum = max(i, itemnum)
    User[u].append(i)

12120726it [00:21, 574424.82it/s]


In [12]:
user_check = list()
for u, l in User.items():
    if len(l) < 5:
        user_check.append(u)
len(user_check)

## 원래 소비 기록 횟수가 5회 이상인 사람들이 있었는데, 아이템 중 소비기록 횟수가 5회가 안 되는 것들이 제외되면서
## 해당 사용자의 소비기록에서도 제외되어서 소비기록이 5회 미만인 사용자들이 생겼다.
## 그래서 train의 최소 길이는 5로 설정해야 한다.

260393

In [2]:
!python3 /home/doyooni303/experiments/LLMRec/ReLLMRec/src/dataset/get_similar_users.py --path /home/doyooni303/experiments/LLMRec/data/amazon/Books 

Books.txt exists
splitting data by user: 100%|█████| 1188598/1188598 [00:03<00:00, 314041.36it/s]
Initializing UserSimilarityFinder...
Finding similar users for 928205 query users...
Processing batches: 100%|█████████████████| 1813/1813 [1:02:05<00:00,  2.06s/it]
Saving results...
All completed !


Train, valid, test txt 파일 저장

In [1]:
import os,sys
sys.path.append("/home/doyooni303/experiments/LLMRec/ReLLMRec/src/dataset")

from get_similar_users import pruning
from utils import data_partition

train, valid, test = data_partition("Books", "/home/doyooni303/experiments/LLMRec/data/amazon/Books")
train = pruning(train, 5, 15)


Preprocessing history: 100%|██████████| 928205/928205 [00:00<00:00, 1228961.78it/s]


In [1]:
import sys
sys.path.append("/home/doyooni303/experiments/LLMRec/ReLLMRec/src")
from dataset.utils import save_SELFRec_data

save_SELFRec_data("Books", "/home/doyooni303/experiments/LLMRec/data/amazon/Books")

100%|██████████| 928205/928205 [00:03<00:00, 278313.18it/s]


Saved SELFRec data to /home/doyooni303/experiments/LLMRec/data/amazon/Books
